In [38]:
import collections

from ortools.sat.colab import visualization
from ortools.sat.python import cp_model


def jssp(durations: list[list[int]], machines: list[list[int]]):
    """Solves the ft06 jobshop."""
    # Creates the solver.
    model = cp_model.CpModel()

    machines_count = len(machines[0])
    jobs_count = len(machines)
    all_machines = range(0, machines_count)
    all_jobs = range(0, jobs_count)

    # Tổng thời gian hoàn thành của tất cả các task
    horizon = sum([sum(durations[i]) for i in all_jobs])

    # Tạo một kiểu dữ liệu mới có tên là task_type, dùng để lưu trữ thông tin của mỗi task
    task_type = collections.namedtuple("task_type", "start end interval")

    """
    Ràng buộc thời gian bắt đầu và kết thúc của mỗi task.
    Điều kiện : 
        - Thời gian bắt đầu và kết thúc của mỗi task phải thỏa mãn điều kiện : 0 <= start <= end <= horizon
        -  start + duration = end
    """
    all_tasks = {}
    for i in all_jobs:
        for j in all_machines:
            start_var = model.NewIntVar(0, horizon, "start_%i_%i" % (i, j))
            duration = durations[i][j]
            end_var = model.NewIntVar(0, horizon, "end_%i_%i" % (i, j))
            interval_var = model.NewIntervalVar(
                start_var, duration, end_var, "interval_%i_%i" % (i, j)
            )
            all_tasks[(i, j)] = task_type(
                start=start_var, end=end_var, interval=interval_var
            )

    """
    Ràng buộc mỗi máy chỉ được thực hiện một task tại một thời điểm.
    """
    machine_to_jobs = {}
    for i in all_machines:
        machines_jobs = []
        for j in all_jobs:
            for k in all_machines:
                if machines[j][k] == i:  # Tìm tất cả các task cần thực hiện trên máy i
                    machines_jobs.append(all_tasks[(j, k)].interval)
        machine_to_jobs[i] = machines_jobs
        # Mỗi máy chỉ được thực hiện một task tại một thời điểm (các khoảng thời gian không được chồng lấn lên nhau)
        model.AddNoOverlap(machines_jobs)

    """
    Ràng buộc tuần tự công nghệ: 
        Với mỗi task của job, thời gian bắt đầu của task tiếp theo phải >= thời gian kết thúc của task trước đó.
    """
    for i in all_jobs:
        for j in range(0, machines_count - 1):
            model.Add(all_tasks[(i, j + 1)].start >= all_tasks[(i, j)].end)

    """
    Mục tiêu cần tối ưu:
        Tìm thời gian hoàn thành của job cuối cùng (makespan) là nhỏ nhất.
    """
    obj_var = model.NewIntVar(0, horizon, "makespan")
    model.AddMaxEquality(
        obj_var, [all_tasks[(i, machines_count - 1)].end for i in all_jobs]
    )  # Thời gian lớn nhất để hoàn thành task cuối cùng của tất cả các job chính là thời gian hoàn thành tất cả các jobs (makespan)
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    status = solver.Solve(model)

    # Output solution.
    if status == cp_model.OPTIMAL:
        starts = [
            [solver.Value(all_tasks[(i, j)][0]) for j in all_machines] for i in all_jobs
        ]
        visualization.DisplayJobshop(starts, durations, machines, "FT")
        print(starts)
        print("Optimal makespan: %i" % solver.ObjectiveValue())
        # Print sequences of jobs assigned to each machine.
        starts = [
            [(starts[i][j], machines[i][j]) for j in all_machines] for i in all_jobs
        ]
        starts = [sorted(job, key=lambda x: x[1]) for job in starts]
        job_sq = []
        for start in zip(*starts):
            start_job = sorted(
                [(i, j[0]) for i, j in enumerate(start)], key=lambda x: x[1]
            )
            job_sq.append([i[0] for i in start_job])
        print()
        print("Optimal job sequence: ")
        for i in job_sq:
            print(*i)


if __name__ == "__main__":
    import numpy as np

    np.random.seed(24)
    # generate random data for durations and machines
    n_job = 10
    n_machine = 11
    durations = np.random.randint(1, 15, size=(n_job, n_machine)).tolist()
    machines = [np.random.permutation(n_machine).tolist() for _ in range(n_job)]
    print("Durations: ")
    for duration in durations:
        print(*duration)
    print("Machines: ")
    for machine in machines:
        print(*machine)
    jssp(durations, machines)

Durations: 
3 4 1 8 2 2 2 5 12 5 4
3 12 12 4 4 8 10 3 8 8 10
4 14 13 1 13 2 2 13 8 7 2
2 7 2 7 10 11 4 1 11 12 2
2 4 5 2 11 10 9 14 8 6 13
4 11 9 10 1 12 6 3 12 5 7
14 1 9 7 3 12 6 8 5 4 12
8 9 4 7 10 8 8 11 11 10 12
1 12 11 2 7 1 11 1 3 1 11
9 7 13 12 13 6 11 13 4 5 6
Machines: 
1 8 5 0 7 10 2 3 9 4 6
3 6 7 1 4 10 9 5 8 2 0
2 10 9 7 8 0 1 4 3 5 6
7 0 9 10 3 6 2 8 4 5 1
10 7 0 3 4 5 2 1 6 8 9
3 7 5 9 8 0 6 2 4 10 1
10 5 4 9 3 8 7 1 0 6 2
7 1 4 8 0 3 10 9 5 2 6
8 9 7 2 5 0 6 3 4 1 10
7 10 1 2 3 4 9 0 5 8 6


[[0, 3, 7, 9, 35, 40, 51, 76, 93, 108, 113], [0, 3, 37, 49, 53, 57, 71, 83, 86, 94, 107], [0, 4, 18, 34, 35, 48, 53, 57, 81, 89, 99], [0, 2, 13, 42, 49, 59, 70, 75, 82, 96, 108], [0, 19, 23, 28, 30, 44, 54, 63, 87, 95, 105], [3, 23, 34, 43, 53, 54, 70, 76, 96, 108, 113], [26, 43, 44, 53, 60, 63, 75, 81, 89, 95, 102], [2, 10, 19, 23, 30, 40, 49, 60, 72, 83, 101], [0, 1, 49, 63, 65, 72, 76, 89, 93, 96, 97], [10, 19, 26, 39, 63, 76, 82, 94, 108, 112, 117]]
Optimal makespan: 123

Optimal job sequence: 
3 0 4 7 2 5 8 6 9 1
0 7 9 1 2 4 6 8 3 5
2 9 0 4 8 3 5 7 1 6
1 5 4 7 3 6 9 0 2 8
7 4 6 1 2 9 3 8 5 0
0 5 6 4 8 7 1 2 3 9
1 3 5 8 4 6 2 7 0 9
3 7 9 4 5 2 0 1 8 6
8 0 7 2 5 6 3 1 4 9
8 3 2 5 6 7 1 9 0 4
4 2 9 6 0 3 7 1 8 5
